<a href="https://colab.research.google.com/github/jbloewencolon/Creating-Dataset-for-The-Demographics-of-Faerun/blob/main/connecting_spatial_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import geopandas as gpd

# Load the shapefiles
cities = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/DnD/Map Files/points.shp")
regions = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/DnD/Map Files/polygons.shp")

# Ensure both are using the same CRS (Coordinate Reference System)
cities = cities.to_crs(regions.crs)

# Perform the spatial join
joined = gpd.sjoin(cities, regions, how="inner", op="within")

# The resulting GeoDataFrame (joined) now has information about which city belongs to which region
print(joined[['NAME', 'Name']])


                NAME              Name
0             Bremen      Icewind Dale
1         Lonelywood      Icewind Dale
2       Caer-Dineval      Icewind Dale
3         Termalaine      Icewind Dale
4             Targos      Icewind Dale
..               ...               ...
657    Ulf of Thuger  The Purple Rocks
658        Vilkstead  The Purple Rocks
659          Gunbarg         Gundarlun
729          Everska          Evereska
730  The Halfway Inn          Evereska

[811 rows x 2 columns]


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
# Save the joined data to a CSV file
joined.to_csv('/content/drive/MyDrive/Colab Notebooks/DnD/joined_data.csv', index=False)

In [ ]:
# 1. Load both datasets
fictional_data = pd.read_excel('ficional_data.xlsx')
city_region_updates = pd.read_csv('city_region_updates.csv')

# 2. Merge datasets based on the 'settlement' column
merged = fictional_data.merge(city_region_updates[['settlement', 'region_kingdom', 'size']],
                              on='settlement',
                              how='left',
                              suffixes=('', '_update'))

# Identify the unmatched rows
unmatched = city_region_updates[~city_region_updates['settlement'].isin(merged['settlement'])]

# 3. Update the columns based on the merge
merged['region_kingdom'] = merged['region_kingdom_update'].combine_first(merged['region_kingdom'])
merged.drop('region_kingdom_update', axis=1, inplace=True)

# 4. Add unmatched rows to the merged dataset
merged = pd.concat([merged, unmatched], ignore_index=True)

# 5. Notify the user about unmatched rows
print(f"{len(unmatched)} unmatched rows from 'city_region_updates.csv' were added to the merged dataset.")

# 6. Save the merged dataset
merged.to_csv('updated_dataset.csv', index=False)
